In [1]:
import pytrends
import pandas as pd
import numpy as np
import time

In [2]:
# import pytrends Package
# props: https://github.com/GeneralMills/pytrends
from pytrends.request import TrendReq

In [3]:
#Read the file
dir = "C:\\Users\\Saurabh\\Downloads\\"
source_file = pd.read_excel(dir + "companies_saurabh.xlsx")

In [4]:
#Just some preprocessing
source_file.rename(columns = {
    'IPO_DATE_TRUE':'Date'}, inplace=True)
source_file= source_file.drop(columns=['Stock Symbol'])
#len(source_file)

In [5]:
source_file['Date'] = pd.to_datetime(source_file['Date'], format='%Y-%m-%d')
source_file['Year'] = source_file['Date'].apply(lambda x:x.strftime('%Y'))
source_file['Month'] = source_file['Date'].apply(lambda x:x.strftime('%m'))
source_file['Day'] = source_file['Date'].apply(lambda x:x.strftime('%d'))
print(source_file.head(5))
#Getting Google Trend values in col
source_file['Year_before_IPO'] = 0
source_file['Month_before_IPO'] = 0
source_file['Day_before_IPO'] = 0

  Organization Name       Date  Year Month Day
0           Twitter 2013-11-09  2013    11  09
1            Square 2015-11-21  2015    11  21
2          Facebook 2012-05-20  2012    05  20
3        ServiceNow 2012-07-22  2012    07  22
4           Shopify 2015-05-23  2015    05  23


In [6]:
source_file[(source_file.Day == '29') & (source_file.Month == '02')] # Checking for leap year Feb

,Organization Name,Date,Year,Month,Day,Year_before_IPO,Month_before_IPO,Day_before_IPO
450,Passage Bio,2020-02-29,2020,02,29,0,0,0


In [7]:
source_file[(source_file.Day == '1') & (source_file.Month == '03')] # Checking for leap year months prior to that

,Organization Name,Date,Year,Month,Day,Year_before_IPO,Month_before_IPO,Day_before_IPO


In [ ]:
kw_list = [i for i in source_file['Organization Name']][:10]

j = 0                                    # For Initilazing first time with data to df and get indices

not_data = []                            # List for not having any companies data 

for i in kw_list:                       
    
    # try and catch block introduced to catch the error and get the list of companies
    try:
        pytrend = TrendReq(hl='en-US',tz=360, timeout=(10,25), proxies=['https://34.203.233.13:80',], retries=2, backoff_factor=0.1)

        #pytrend.build_payload(kw_list = [i])       # Build Payloads
        
        day,month,year = int(source_file.Day[j]),int(source_file.Month[j]),int(source_file.Year[j]) #intitialising day month and year

        
        # Checking for consistency as there could be leap year with 29 days in Feb
        if day == 29 and year == 2016:
            day_old = 28
            
        else:
             day_old = day
            
        
        # Year Before IPO let's check the trends
        year_before_IPO = pytrend.get_historical_interest([i], year_start=year - 1,
                                         month_start=month, day_start=day_old, year_end=year,
                                         month_end=month, day_end=day,
                                         cat=0, geo='', gprop='', sleep=0)      # Build Payloads
        source_file['year_before_IPO'][j] = np.mean(year_before_IPO['' + i])
        
        
        if month == 1:
            month_old = 12
            year_old = year - 1
        
        else:
            month_old = month - 1 
            year_old = year
        
        # If it's march saves the leap year
        if day == 31 and month == 3:
            day_old = 28
            
        else:
             day_old = day
                
        # Month Before IPO let's check the trends 
        month_before_IPO = pytrend.get_historical_interest([i], year_start=year_old,
                                         month_start=month_old, day_start=day_old, year_end=year,
                                         month_end=month, day_end=day,
                                         cat=0, geo='', gprop='', sleep=0)      # Build Payloads
        source_file['month_before_IPO'][j] = np.mean(month_before_IPO['' + i])
        
        
        if day == 1:
            day_old = 30
            
        else:
            day_old = day - 1
        # Day Before IPO let's check the trends 
        day_before_IPO = pytrend.get_historical_interest([i], year_start=year,
                                         month_start=month, day_start=day_old, year_end=year,
                                         month_end=month, day_end=day,
                                         cat=0, geo='', gprop='', sleep=0)      # Build Payloads
        source_file['day_before_IPO'][j] = np.mean(day_before_IPO['' + i])

        j = j + 1
        
    except:
        
        print("Not Possible for %s company"%i)
        not_data.append(i)                     # Appending if not possible and continue
        j = j + 1
        
        
    if(j % 10 == 100):
        print("Done for %d companies and %d companies are not there in list"%(j,len(not_data)))
        continue
        
print("Done for all")        
print("Done for %d companies and %d companies are not there in list"%(j,len(not_data)))        
print(source_file)

Proxy error. Changing IP


In [ ]:
source_file.to_csv(dir + 'Google_Trend_Companies.csv', encoding='utf_8_sig')






# Redundant API Dumps
dir = "C:\\Users\\Saurabh\\Desktop\\"

def Google_Trend_API(kw_list,dir):
    """
    This Function takes a list and directory as input for finding the trend data from Google weekly for last 5 years
    
    Input : Input to the function is list of companies  and directory to store  API
    
    Output: Output of the funtion is .csv file of companies and a list whose data couldn't be extracted
    
    """
    from pytrends.request import TrendReq    # Importing Libraries

    i = 0                                    # For Initilazing first time
    
    df = pd.DataFrame()                      # For Storing the values
    
    not_data = []                            # List for not having any companies data 
    
    for i in kw_list:                              # For Initializing the list
        
    # try and catch block introduced to catch the error and get the list of companies
        try:
            pytrend = TrendReq(hl='en-US', timeframe='all',tz=360)

            pytrend.build_payload(kw_list = [i])       # Build Payloads

            data = pytrend.interest_over_time()  # Generates the data

            # Storing in dataframe
            df['' + i] = data[i]
            #print(df)


        except:
            print("Not Possible for %s company"%i)
            not_data.append(i)                     # Appending if not possible and continue
            continue

    print("Could not get data for following list of companies:")
    print(not_data)

        # Storing to csv file
    df.to_csv(dir + 'Google_Trend_Company.csv', encoding='utf_8_sig')

    return not_data

# User list to be defined
kw_list = ['Despegar', 'MercadoLibre', 'Grupo Netshoes', 'Azul SA',
       'Stone Pagamentos SA', 'Arco Educacao', 'Canacol Energy',
       'IRSA Propiedades Comerciales', 'Sienna Biopharmaceuticals',
       'Kior', 'Harpoon Therapeutics', 'ARYx Therapeutics',
       'WPCS International', 'Kips Bay Medical', 'Intertainment Media',
       'Sonus Networks', 'Stratos Lightwave',
       'Advanced Life Sciences Holdings', 'InterVideo', 'StorageNetworks',
       'Anesiva', 'Helicos BioSciences', 'Evergreen Solar', 'eMachines',
       'Penson Worldwide', 'iBEAM Broadcasting',
       'Transcept Pharmaceuticals', 'Soapstone Networks',
       'Crude Carriers Corp.', 'MaSpaces', 'Novartis', 'Delivery Hero',
       'Partners Group', 'Spotify', 'Farfetch', 'HelloFresh',
       'Funding Circle', 'Mail.Ru Group', 'Draper Esprit', 'Takeaway.com',
       'Rocket Internet', 'Storytel', 'Mimecast', 'IP Group Plc',
       'Immotion Group plc', 'trivago', 'Stillfront Group',
       'Marley Spoon', 'FunCom', 'Abcam']

# Please give directory
d = "C:\\Users\\Saurabh\\Desktop\\"

no_company = Google_Trend_API(kw_list,d)
              